In [1]:
# Setup redis
%pip install redis

Note: you may need to restart the kernel to use updated packages.


In [3]:
fullName = "Sarah Smalley" # <--- PUT YOUR FULL NAME HERE (e.g. Jimmy Jones)

assert fullName != "Full Name"

password = input("Enter the password displayed on the projector in class! \nPassword: ")
print(f"You entered: {password}")

You entered: jumpJUMP1!


In [4]:
# Connect

import redis
import time

host = 'redis-15949.c251.east-us-mz.azure.redns.redis-cloud.com'
port = 15949
username = 'CS452'

r = redis.Redis(host=host, port=port, password=password, db=0, username=username)

In [5]:
# set

r.set(fullName, 'hello world') # True

True

In [6]:
# get
result = r.get(fullName)

print(result.decode())

hello world


In [7]:
# set with expiry!

myKey = 'expiringKey_' + fullName
r.set(myKey, "I only get to be here for a short time", ex=3)

start = time.time()
while result := r.get(myKey):
    print(result, time.time() - start, myKey)

print("result gone!")

b'I only get to be here for a short time' 0.10586333274841309 expiringKey_Sarah Smalley
b'I only get to be here for a short time' 0.16683506965637207 expiringKey_Sarah Smalley
b'I only get to be here for a short time' 0.31306028366088867 expiringKey_Sarah Smalley
b'I only get to be here for a short time' 0.40820980072021484 expiringKey_Sarah Smalley
b'I only get to be here for a short time' 0.4742105007171631 expiringKey_Sarah Smalley
b'I only get to be here for a short time' 0.6199929714202881 expiringKey_Sarah Smalley
b'I only get to be here for a short time' 0.6819984912872314 expiringKey_Sarah Smalley
b'I only get to be here for a short time' 0.8271567821502686 expiringKey_Sarah Smalley
b'I only get to be here for a short time' 0.9289631843566895 expiringKey_Sarah Smalley
b'I only get to be here for a short time' 1.0306808948516846 expiringKey_Sarah Smalley
b'I only get to be here for a short time' 1.1347544193267822 expiringKey_Sarah Smalley
b'I only get to be here for a short tim

In [8]:
# delete

r.delete(fullName)
ans = r.get(fullName)

if ans == None:
    print(f'key "{fullName}" is not there anymore.')
else:
    print(f'key "{fullName}" is {ans.decode()}')

key "Sarah Smalley" is not there anymore.


In [9]:
# sets! and sismember

setKey = "iwashere"
print("Did you sign the guest book?")
print("yes" if r.sismember(setKey, fullName) else "no")

Did you sign the guest book?
no


In [11]:
# sadd

r.sadd(setKey, fullName)

0

In [12]:
# sismember

print("Did you sign the guest book?")
print("yes" if r.sismember(setKey, fullName) else "no")

Did you sign the guest book?
yes


In [13]:
# scard and smembers

print(f"We have {r.scard(setKey)} {'entry' if r.scard(setKey) == 1 else 'entries'}!")
print(r.smembers("iwashere"))

We have 17 entries!
{b'Madeline Evans', b'Sarah Smalley', b'Michael Reynolds', b'Patrick Bennett', b'Connor Barrett', b'Bentley Bigelow', b'Micaela Madariaga', b'Ethan Alvey', b'Elijah Thompson', b'Rebecca Ogden', b'Peter May', b'Bryce Lasson', b'Santiago Bolano', b'Grant Alston', b'Wesley dos Santos', b'Connor Barret', b'Yuto Kikuchi'}


In [59]:
# incr

print("ring the bell!")
for i in range (0,100):
  result = r.incr("bell", 5000)

print(f"This was the ring number {result}!")

# When would this be useful? Inventory?

ring the bell!
This was the ring number -3599999999999499999!


In [61]:
# Unsafe Transfers..

for _ in range(4):
    print("--- LOOP ---")

    # Starting amounts
    print(f'Starting: fred$ = {r.get("fredsMoney")}, george$ = {r.get("georgesMoney")}')

    # Unsafe transfer $100
    # race condition here
    fredsMoney = r.get("fredsMoney")
    fredsMoney = r.set("fredsMoney", int(fredsMoney.decode()) - 100)

    georgesMoney = r.get("georgesMoney")
    georgesMoney = r.set("georgesMoney", int(georgesMoney.decode()) + 100)

    print(f'Transfered: fred$ = {r.get("fredsMoney")}, george$ = {r.get("georgesMoney")}')

    # Unsafe transfer the money back!
    fredsMoney = r.get("fredsMoney")
    fredsMoney = r.set("fredsMoney", int(fredsMoney.decode()) + 100)

    georgesMoney = r.get("georgesMoney")
    georgesMoney = r.set("georgesMoney", int(georgesMoney.decode()) - 100)

    # Ending amounts
    print(f'Transfered back: fred$ = {r.get("fredsMoney")}, george$ = {r.get("georgesMoney")}')

--- LOOP ---
Starting: fred$ = b'11600', george$ = b'11600'
Transfered: fred$ = b'11500', george$ = b'11700'
Transfered back: fred$ = b'11600', george$ = b'11600'
--- LOOP ---
Starting: fred$ = b'11600', george$ = b'11600'
Transfered: fred$ = b'11500', george$ = b'11700'
Transfered back: fred$ = b'11600', george$ = b'11600'
--- LOOP ---
Starting: fred$ = b'11600', george$ = b'11600'
Transfered: fred$ = b'11500', george$ = b'11700'
Transfered back: fred$ = b'11600', george$ = b'11600'
--- LOOP ---
Starting: fred$ = b'11600', george$ = b'11600'
Transfered: fred$ = b'11500', george$ = b'11700'
Transfered back: fred$ = b'11600', george$ = b'11600'


In [ ]:
# Fix Fred and George's accounts
r.set("fredsMoney", 10000)
r.set("georgesMoney", 10000)

In [64]:
# Transfer with locking!  set(ex and nx = True)

# Safe Transfers..
safeTransferLockName = "cs452PatienceLock"
def getLock():
    print("Waiting for lock...", end="")
    while True:
        if r.set(safeTransferLockName, "MINE!", ex=2, nx=True):
            print()
            print("SET LOCK")
            return
        print(".", end="")
        time.sleep(.01)

def releaseLock():
    print("RELEASED LOCK")
    r.delete(safeTransferLockName)


for _ in range(4):
    print("--- LOOP ---")
    getLock()  # <--------- GET THE LOCK ----------------<<

    # Starting amounts
    print(f'Staring: fred$ = {r.get("fredsMoney")}, george$ = {r.get("georgesMoney")}')


    # Safe transfer $100
    fredsMoney = r.get("fredsMoney")
    fredsMoney = r.set("fredsMoney", int(fredsMoney.decode()) - 100)

    georgesMoney = r.get("georgesMoney")
    georgesMoney = r.set("georgesMoney", int(georgesMoney.decode()) + 100)

    print(f'Transfered: fred$ = {r.get("fredsMoney")}, george$ = {r.get("georgesMoney")}')

    # Safe transfer the money back!
    fredsMoney = r.get("fredsMoney")
    fredsMoney = r.set("fredsMoney", int(fredsMoney.decode()) + 100)

    georgesMoney = r.get("georgesMoney")
    georgesMoney = r.set("georgesMoney", int(georgesMoney.decode()) - 100)

    # Ending amounts
    print(f'Transfered back: fred$ = {r.get("fredsMoney")}, george$ = {r.get("georgesMoney")}')

    releaseLock() # <------------ RELEASE THE LOCK -------------------<<

--- LOOP ---
Waiting for lock...
SET LOCK
Staring: fred$ = b'10000', george$ = b'10000'
Transfered: fred$ = b'9900', george$ = b'10100'
Transfered back: fred$ = b'10000', george$ = b'10000'
RELEASED LOCK
--- LOOP ---
Waiting for lock.......
SET LOCK
Staring: fred$ = b'10000', george$ = b'10000'
Transfered: fred$ = b'9900', george$ = b'10100'
Transfered back: fred$ = b'10000', george$ = b'10000'
RELEASED LOCK
--- LOOP ---
Waiting for lock....................................................................................................................................................................................................................................................
SET LOCK
Staring: fred$ = b'10000', george$ = b'9500'
Transfered: fred$ = b'9900', george$ = b'9500'
Transfered back: fred$ = b'9800', george$ = b'9400'
RELEASED LOCK
--- LOOP ---
Waiting for lock......................
SET LOCK
Staring: fred$ = b'10000', george$ = b'9300'
Transfered: fred$ = b'9800', george$ = b'

In [68]:
# Barriers and work queues
barrierKey = "everyOneIsWaitingForThis"


In [ ]:
# Restore the work queue
r.rpush("serviceQueueKey", f"Wash dishes for {fullName}")
r.rpush("serviceQueueKey", f"Cook dinner for {fullName}")
r.rpush("serviceQueueKey", *[f"Service Opportunity for ({fullName})" for _ in range(3)])

965

In [70]:
# Barrier
timeout = time.time() + 120 # two minutes
def waitOnBarrier():
    print("Wait for barrier...")
    while not r.get(barrierKey):
        print(".", end="")
        time.sleep(.01)
        if time.time() > timeout:
            raise TimeoutError("Waiting too long for barrier! Try again?")

    print()
    print("Go!")

waitOnBarrier()

# Pull work off of work queue!
for _ in range(10):
    result = r.lpop("serviceQueueKey")
    print(result)

Wait for barrier...
................................................................................
Go!
b'Wash dishes for Sarah Smalley'
b'Service Opportunity for (Santiago Bolano)'
b'Service Opportunity for (Yuto Kikuchi)'
b'Wash dishes for Sam Jones'
b'Cook dinner for Sam Jones'
b'Service Opportunity for (Sam Jones)'
b'Service Opportunity for (Sam Jones)'
b'Service Opportunity for (Sam Jones)'
b'Wash dishes for Wesley dos Santos'
b'Cook dinner for Wesley dos Santos'


In [ ]:
# Allow Barrier
r.set(barrierKey, "Let them go!", ex=5)

In [ ]:
# Block Barrier
r.delete(barrierKey) # Should be an expiring key already, but just in case someone put something else in :)